In [85]:
import pandas as pd
import textstat as tst
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

In [86]:
stop_words = set(stopwords.words('english')) 

In [98]:
df = pd.read_csv("complexo.csv")

In [99]:
letras = [[letra for letra in df['letras_limpas'][n].split() if letra not in stop_words] for n in range(len(df))]

In [100]:
model = Word2Vec(letras, vector_size=100, window=5, min_count=1, workers=4)  # Treina o modelo Word2Vec

In [90]:
def calcular_variancia_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:  # Verifica se a palavra está no vocabulário do modelo
            vetores.append(modelo.wv[palavra])  # Retorna o vetor da palavra
    
    if len(vetores) > 0:  # Evitar calcular variância em lista vazia
        vetores_np = np.array(vetores)  # Converte a lista de vetores em um array NumPy

        scaler = MinMaxScaler()
        vetores_normalizados = scaler.fit_transform(vetores_np)
        
        variancia = np.var(vetores_normalizados, axis=0)  # Calcula a variância ao longo do eixo 0
        
        return np.sum(variancia)  # Retorna a soma da variância
        
    else:
        
        return 0  # Retorna 0 se não houver vetores


In [76]:
def avaliar_complexidade_letra(letra, modelo):
    # Remove stopwords e processa a letra
    letra_processada = [palavra.lower() for palavra in letra.split() if palavra not in stop_words]
    return calcular_variancia_vetores(letra_processada, modelo)

In [77]:
letra = df.letras_limpas[0]
complexidade = avaliar_complexidade_letra(letra, model) 

In [78]:
complexidade

5.0631924

In [101]:
themes = {
    'love': ['love', 'affection', 'heart', 'passion', 'desire', 'romance'],
    'sadness': ['sadness', 'pain', 'longing', 'loneliness', 'lament', 'loss'],
    'freedom': ['freedom', 'live', 'free', 'independence', 'choice', 'escape'],
    'friendship': ['friendship', 'companion', 'loyalty', 'trust', 'camaraderie'],
    'hope': ['hope', 'faith', 'dream', 'future', 'light', 'renewal'],
    'nature': ['nature', 'earth', 'sky', 'sea', 'forest', 'wildlife'],
    'struggle': ['struggle', 'battle', 'strength', 'resistance', 'challenge', 'conquest'],
    'joy': ['joy', 'happiness', 'laughter', 'smile', 'celebration', 'fun'],
    'solitude': ['solitude', 'isolation', 'distance', 'disconnection', 'emptiness'],
    'nostalgia': ['nostalgia', 'memories', 'past', 'remembrance', 'reminiscence'],
    'change': ['change', 'transformation', 'growth', 'evolution', 'new beginnings'],
    'self-love': ['self-love', 'self-esteem', 'self-awareness', 'acceptance', 'confidence'],
    'conflict': ['conflict', 'dispute', 'disagreement', 'tension', 'polarization'],
    'solidarity': ['solidarity', 'help', 'support', 'community', 'union'],
    'socio-political': ['socio-political', 'justice', 'rights', 'freedom of speech', 'equality'],
    'party': ['party', 'celebration', 'fun', 'dance', 'joy'],
    'mental health': ['mental health', 'anxiety', 'depression', 'healing', 'balance'],
}
def normalizar_similaridade(similaridade, vetor_letra):
    # Normaliza pela magnitude do vetor letra
    magnitude_letra = np.linalg.norm(vetor_letra)
    
    if magnitude_letra != 0:
        return similaridade / magnitude_letra
    return similaridade

def media_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:
            vetores.append(modelo.wv[palavra])
    
    if len(vetores) == 0:
        return np.zeros(modelo.vector_size)  # Retorna um vetor de zeros se não houver palavras válidas
    
    return np.mean(vetores, axis=0)

def resultado(themes, model, vetor_letra):
    resultado = {}
    for tema, palavras_chave in themes.items():
        vetor_tema = media_vetores(palavras_chave, model)

        if vetor_tema is not None and vetor_letra is not None and np.any(vetor_tema) and np.any(vetor_letra):
            similaridade = model.wv.cosine_similarities(vetor_letra, np.array([vetor_tema]))
            
            # Aplica a normalização da similaridade
            similaridade_normalizada = normalizar_similaridade(similaridade[0], vetor_letra)
            resultado[tema] = similaridade_normalizada
        else:
            resultado[tema] = 0
    
    return resultado

letra = df.letras_limpas[2]    
    
letra_processada = [palavra.lower() for palavra in letra.split() if palavra.lower() not in stop_words]
    
vetor_letra = media_vetores(letra_processada, model)
    
    
res = resultado(themes, model, vetor_letra)


In [84]:
res

{'love': 0.09862992,
 'sadness': 0.0690857,
 'freedom': 0.090203345,
 'friendship': 0.08698643,
 'hope': 0.09112468,
 'nature': 0.045271896,
 'struggle': 0.05093371,
 'joy': 0.09118261,
 'solitude': 0.044652026,
 'nostalgia': 0.054162756,
 'change': 0.07409487,
 'self-love': 0.03900786,
 'conflict': 0.010073365,
 'solidarity': 0.06763613,
 'socio-political': 0.009180461,
 'party': 0.07584023,
 'mental health': 0.039802894}

In [96]:
res

{'love': 0.10065905,
 'sadness': 0.06550171,
 'freedom': 0.08896069,
 'friendship': 0.09403873,
 'hope': 0.092761986,
 'nature': 0.038707662,
 'struggle': 0.049240526,
 'joy': 0.09025223,
 'solitude': 0.0417635,
 'nostalgia': 0.054071944,
 'change': 0.080175325,
 'self-love': 0.032585975,
 'conflict': 0.023946512,
 'solidarity': 0.06704344,
 'socio-political': 0.006699286,
 'party': 0.07927788,
 'mental health': 0.03656185}